In [1]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# change the paths with respect to the local computer path for Josh
! cd  /content/gdrive/MyDrive/DataScience_Project

In [ ]:
! git clone https://github.com/joshuaaduarte/categorize_ethnicity.git

Cloning into 'categorize_ethnicity'...
remote: Enumerating objects: 97563, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 97563 (delta 0), reused 3 (delta 0), pack-reused 97560
Receiving objects: 100% (97563/97563), 536.92 MiB | 13.43 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (97704/97704), done.


In [1]:
# import the libraries as shown below
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
pwd

'c:\\Berkeley\\spring2023\\sdse\\categorize_ethnicity'

In [3]:
# change the paths with respect to the local computer path for Josh
IMAGE_SIZE = [224,224]
train_path = "C:\\Berkeley\\spring2023\\sdse\\categorize_ethnicity\\train"
test_path = "C:\\Berkeley\\spring2023\\sdse\\categorize_ethnicity\\val"

In [4]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [6]:
folders = glob("C:\\Berkeley\\spring2023\\sdse\\categorize_ethnicity\\train\\*")

In [7]:
folders

['C:\\Berkeley\\spring2023\\sdse\\categorize_ethnicity\\train\\Black',
 'C:\\Berkeley\\spring2023\\sdse\\categorize_ethnicity\\train\\East Asian',
 'C:\\Berkeley\\spring2023\\sdse\\categorize_ethnicity\\train\\Indian',
 'C:\\Berkeley\\spring2023\\sdse\\categorize_ethnicity\\train\\Latino_Hispanic',
 'C:\\Berkeley\\spring2023\\sdse\\categorize_ethnicity\\train\\Middle Eastern',
 'C:\\Berkeley\\spring2023\\sdse\\categorize_ethnicity\\train\\Southeast Asian',
 'C:\\Berkeley\\spring2023\\sdse\\categorize_ethnicity\\train\\White']

In [8]:
len(folders)

7

In [9]:
x = Flatten()(vgg16.output)

In [10]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   rotation_range=2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 9181 images belonging to 7 classes.


In [15]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 4844 images belonging to 7 classes.


In [16]:
import matplotlib.pyplot as plt

In [17]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=1,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

: 

: 

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')